# SQL Injection

In [11]:
import psycopg2             # python->psql connection
import psycopg2.extras
import pandas as pd         # create dataframes 
# Import the 'config' function from the config_user_dta.py file:
from config import config

# Get the config params
params_ = config()
# Connect to the Postgres_DB:
conn = psycopg2.connect(**params_)
# Create new_cursor allowing us to write Python to execute PSQL:
cur = conn.cursor()
conn.autocommit = True

In [12]:
# CREATE TABLE FOR PSQL: staging_fake_ppl

def create_staging_table(cursor):
    cursor.execute("""
        DROP TABLE IF EXISTS staging_website_users;
        CREATE UNLOGGED TABLE staging_website_users (
            user_          varchar(30),
            DB_Admin       Boolean
        );""")

# Send the Schema to PSQL
with conn.cursor() as cursor:
    create_staging_table(cursor)


In [13]:
# Insert some fake data:
sql_="""INSERT INTO staging_website_users (user_,DB_Admin) VALUES ('Alf',True),
('Monty',False),('humpty_dumpty',False),('yaya123',False);"""
cur.execute(sql_)

# verify
cur.execute("select * from staging_website_users")
cur.fetchall()

[('Alf', True), ('Monty', False), ('humpty_dumpty', False), ('yaya123', False)]

In [14]:
# BAD EXAMPLE. DON'T DO THIS! 
def is_admin(username: str):
    with conn.cursor() as cursor:
        cur.execute("""
            SELECT
                DB_admin
            FROM
                staging_website_users
            WHERE
                user_ = '%s'          
                                    
        """ % username)               #  this line % and '' in line above: NO NO NO
        result = cur.fetchone()

    if result is None:
        # User does not exist
        return False

    admin, = result
    return admin


In [15]:
print(is_admin('Alf'))
print(is_admin('yay123'))

True
False


In [16]:

print('Pirate entered your DB:',is_admin("'; select true; --"))

Pirate entered your DB: True


# GOOD Example (1)

In [17]:
from psycopg2 import sql #help safely write sql queries, verify you can't put bad stuff!

def find_dta(table_name: str, f_name: str,name:str):
    with conn.cursor() as cursor:
        state_ment = sql.SQL("""
            SELECT
                *
            FROM {table_name}
                WHERE {f_name}={name}
        """).format(
            table_name = sql.Identifier(table_name),
            f_name = sql.Identifier(f_name),
            name=sql.Literal(name)
        )
        cur.execute(state_ment)
        result = cur.fetchall()

    return result

In [18]:
pd.DataFrame(find_dta('staging_fake_ppl','first_name','Anthony'),
             columns=['credit_card','email',
                        'first_name','last_name','phone_number']).head()

,credit_card,email,first_name,last_name,phone_number
0,8631-4500-5666-1510,rztkvliou@dkeinhgysf.deo,Anthony,Vo,7345795348
1,1280-4584-5250-1133,axthrlcob@kdqvoabtw.nmg,Anthony,Henley,6958349723
2,6921-1083-1797-8702,xzgefkqlh@nhocyb.qwy,Anthony,Hazelman,9315684987
3,5725-6056-1230-2423,kj@tr.jdt,Anthony,Garcia,7244972846
4,8170-8477-1042-3899,rexi@eitlpqv.pce,Anthony,Green,8362748419


# GOOD Example (2)

In [19]:
# same thing almost:

query = sql.SQL("select * from {table} where {column_ofInterest} = %s").format(
    table=sql.Identifier('staging_fake_ppl'),
    column_ofInterest=sql.Identifier('first_name'))

cur.execute(query,('Anthony',))

pd.DataFrame(cur.fetchall(),columns=['credit_card','email',
                        'first_name','last_name','phone_number']).head()

,credit_card,email,first_name,last_name,phone_number
0,8631-4500-5666-1510,rztkvliou@dkeinhgysf.deo,Anthony,Vo,7345795348
1,1280-4584-5250-1133,axthrlcob@kdqvoabtw.nmg,Anthony,Henley,6958349723
2,6921-1083-1797-8702,xzgefkqlh@nhocyb.qwy,Anthony,Hazelman,9315684987
3,5725-6056-1230-2423,kj@tr.jdt,Anthony,Garcia,7244972846
4,8170-8477-1042-3899,rexi@eitlpqv.pce,Anthony,Green,8362748419


go here for an idea: https://portswigger.net/web-security/sql-injection